# Compound

> Fill in a module description here

In [ ]:
#| default_exp compound.core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from dataclasses import dataclass

from exex.imports import *
from exex.core import *
from exex.system import *
from exex.utils import *

### States

In [ ]:
#| export
@dataclass
class State:
    SOLID = 'solid'
    LIQUID = 'liquid'
    GAS = 'gas'

### Matter

In [ ]:
#| export
class Matter(metaclass=PrePostInitMeta):
    def __init__(self):
        self.properties = dict()
        self.laws = dict()
        self.time: int = None
        self.system = System()
    
    def __post_init__(self, *args, **kwargs): 
        self._setup()

    def _setup_laws(self, laws: list[Law]) -> None: # add laws from `self.add_laws` to compound
        
        for law in laws:
            name = law.snake_name
            
            if not name in self.laws:
                law = law(compound=self)
                law._run_config()
                self.laws[name] = law
    
    def _setup(self) -> None:
        self._setup_laws(self.add_laws)
    
    def _set_system(
        self,
        system: System # the system
    ) -> None:
        self.system = system
    
    def get_system(self):
        return self.system
    
    def set_time(
        self,
        time: int # time
    ):
        self.time = time
        return self

All properties that a compound has always being governed by some laws.

In [ ]:
@patch(as_prop=True)
def property_exists(self: Matter, name):
    return self.name in self.properties

In [ ]:
#| export
@patch
@use_kwargs_dict(unit=None)
def get_prop(
    self: Matter,
    name: str, # name
    t: int, # time
    **kwargs
):
    #if self.property_exists(args['name'])
    return self.system.get_prop(name, t, instance=self, **kwargs)

In [ ]:
#| export
@patch
@use_kwargs_dict(unit=None, eval=False)
def set_prop(self: Matter, name, val, t, **kwargs):
    return self.system.set_prop(name, val, t, instance=self, **kwargs)

#### Laws

In [ ]:
#| export
class MassMoleRatio(Law):
    def __init__(self, compound):
        super().__init__()
        self.compound = compound
        #self.properties = [Mass, Mole, MolarMass]
        self.properties = [
            {"object": Mass},
            {"object": Mole},
        ]
    
    def expr(self):
        return self.compound.get_properties('mass')

### Compound

In [ ]:
#| export
class Compound(Matter):
    
    LAWS = [MassMoleRatio]
    
    def __init__(
        self,
        formula: str # the chemical formula
    ) -> None:
        super().__init__()
        
        compound = chemlib.Compound(formula)
        #self._laws = [MassMoleRatio]
        self.add_laws = [MassMoleRatio]
        
        self.elements = compound.elements
        self.formula = compound.formula
        self._formula = formula
        self.coefficient = compound.coefficient
        self.occurences = compound.occurences
        
        self._setup_laws([MassMoleRatio])
    
    @property
    def snake_name(self) -> str: # return the snake name style
        return self._formula
    
    def info(self, **kwargs):
        dta = {}
        
        for k, v in self.properties.items():
            # data_point = {}
            # print(v._data)
            key = k
            # if v.unit:
            #     key += f' ({v.unit})'
        
            dta[key] = v._data
        
        df = pd.DataFrame(data=dta, **kwargs)
        df.index.name = "Time"
        return df.sort_index()
    
    def get_data(
        self,
        time: int, # the time
        name: str # the property name
    ):
        if not name in self.properties:
            return "The property don't exist"
        pass

    __repr__ = basic_repr('formula')

In [ ]:
@patch
def set_amount(self: Compound, amount):
    pass

In [ ]:
def set_mass():
    pass

In [ ]:
class A:
    COMPOUNDS = [10, 20, 30]
    def __init__(self):
        self.compounds = [1, 2, 3]

In [ ]:
class B(A):
    COMPOUNDS = [40, 50, 60]
    def __init__(self):
        super().__init__()
        self.compounds = [4, 5, 6]
    
    @classmethod
    def base(cls):
        return cls.__bases__

In [ ]:
b = B()

In [ ]:
b.base()

(__main__.A,)

In [ ]:
A.__bases__

(object,)

In [ ]:
B.__bases__

(__main__.A,)

In [ ]:
b.__dict__

{'compounds': [4, 5, 6]}

In [ ]:
H2O = Compound('H2O')

In [ ]:
H2O.__dict__

{'properties': {'mass': <exex.core.Mass>,
  'mole': <exex.core.Mole>},
 'laws': {'mass_mole_ratio': <__main__.MassMoleRatio>},
 'time': None,
 'system': <exex.system.System>,
 'add_laws': [__main__.MassMoleRatio],
 'elements': [<chemlib.chemistry.Element>,
  <chemlib.chemistry.Element>,
  <chemlib.chemistry.Element>],
 'formula': 'H₂O₁',
 '_formula': 'H2O',
 'coefficient': 1,
 'occurences': {'H': 2, 'O': 1}}

In [ ]:
#| hide
test_eq(H2O.occurences['H'], 2)
test_eq(len(H2O.elements), 3)
test_eq(H2O.snake_name, 'H2O')

In [ ]:
#| hide
test_eq(H2O.properties['mass'].unit, Unit.MASS)
test_eq(H2O.properties['mole'].unit, Unit.MOLE)